In [1]:
import pandas as pd
import json
import difflib
from pathlib import Path

# File paths
batter_stats_path = Path("./data_exports/ipl_batter_stats.csv")
bowler_stats_path = Path("./data_exports/ipl_bowler_stats.csv")
players_json_path = Path("./players.json")
output_path = Path("./data_exports/ipl_team_impacts.json")

# Load data
batter_stats = pd.read_csv(batter_stats_path)
bowler_stats = pd.read_csv(bowler_stats_path)
with open(players_json_path, "r") as f:
    player_data = json.load(f)

# Column names
batter_name_col = "Player"
batter_impact_col = "Total Batting Impact"
bowler_name_col = "Bowler"
bowler_impact_col = "Total Bowling Impact"

# Collect player names
batter_names = batter_stats[batter_name_col].dropna().unique().tolist()
bowler_names = bowler_stats[bowler_name_col].dropna().unique().tolist()
all_stat_names = list(set(batter_names + bowler_names))

# Fuzzy match function
def match_name(name, name_list):
    match = difflib.get_close_matches(name, name_list, n=1, cutoff=0.6)
    return match[0] if match else None

# Build impact data
team_impact = {}
for team in player_data["teams"]:
    team_name = team["name"]
    team_impact[team_name] = []

    for player in team["players"]:
        full_name = player["name"]
        matched_name = match_name(full_name, all_stat_names)

        batting_impact = 0
        bowling_impact = 0

        if matched_name:
            if matched_name in batter_names:
                batting_impact = batter_stats.loc[
                    batter_stats[batter_name_col] == matched_name, batter_impact_col
                ].sum()
            if matched_name in bowler_names:
                bowling_impact = bowler_stats.loc[
                    bowler_stats[bowler_name_col] == matched_name, bowler_impact_col
                ].sum()

        team_impact[team_name].append({
            "Player": full_name,
            "Matched Name": matched_name,
            "Batting Impact": batting_impact,
            "Bowling Impact": bowling_impact,
            "Total Impact": batting_impact + bowling_impact
        })

# Save output
with open(output_path, "w") as f:
    json.dump(team_impact, f, indent=2)

print(f"Saved: {output_path}")


Saved: data_exports\ipl_team_impacts.json


In [2]:
import pandas as pd
import json
from pathlib import Path

# Load the previously generated JSON file
input_path = Path("./data_exports/ipl_team_impacts.json")
with open(input_path, "r") as f:
    team_impact_data = json.load(f)

# Calculate total impact per team
team_totals = []
for team, players in team_impact_data.items():
    total = sum(player["Total Impact"] for player in players)
    team_totals.append({"Team": team, "Total Impact": total})

# Create and sort DataFrame
impact_df = pd.DataFrame(team_totals).sort_values(by="Total Impact", ascending=False).reset_index(drop=True)

impact_df

,Team,Total Impact
0,Kolkata Knight Riders,11.240328
1,Chennai Super Kings,10.792871
2,Royal Challengers Bengaluru,10.729772
3,Gujarat Titans,10.470693
4,Mumbai Indians,10.106527
5,Rajasthan Royals,9.541861
6,Delhi Capitals,8.694924
7,Lucknow Super Giants,5.910746
8,Sunrisers Hyderabad,5.880838
9,Punjab Kings,5.185222


In [3]:
# Fix the country lookup logic — refactor to build a name-to-country mapping before selecting squads

# Build a mapping of player full names to their countries
name_to_country = {}
for team in player_data["teams"]:
    for player in team["players"]:
        name_to_country[player["name"]] = player.get("country", "India")

# Updated function with external country lookup
def select_ideal_squad(players):
    sorted_players = sorted(players, key=lambda p: p["Total Impact"], reverse=True)
    squad = []
    foreign_count = 0

    for player in sorted_players:
        full_name = player["Player"]
        country = name_to_country.get(full_name, "India")
        is_foreign = country != "India"

        if is_foreign and foreign_count >= 4:
            continue

        squad.append(player)
        if is_foreign:
            foreign_count += 1

        if len(squad) == 11:
            break

    return squad

# Generate ideal squads and total impact
ideal_squads = []
for team, players in team_impact_data.items():
    squad = select_ideal_squad(players)
    total = sum(p["Total Impact"] for p in squad)
    ideal_squads.append({
        "Team": team,
        "Total Impact": total,
        "Squad": [p["Player"] for p in squad]
    })

# Create display table
ideal_df = pd.DataFrame(ideal_squads).sort_values(by="Total Impact", ascending=False).reset_index(drop=True)
ideal_df

,Team,Total Impact,Squad
0,Gujarat Titans,9.048098,"[Rashid Khan, Rahul Tewatia, Washington Sundar..."
1,Kolkata Knight Riders,8.682876,"[Sunil Narine, Moeen Ali, Andre Russell, Spenc..."
2,Mumbai Indians,8.542635,"[Hardik Pandya, Jasprit Bumrah, Karn Sharma, D..."
3,Royal Challengers Bengaluru,8.509036,"[Abhinandan Singh, Krunal Pandya, Lungi Ngidi,..."
4,Chennai Super Kings,8.405288,"[Ravindra Jadeja, Shreyas Gopal, Matheesha Pat..."
5,Rajasthan Royals,7.876849,"[Jofra Archer, Yudhvir Singh, Sandeep Sharma, ..."
6,Delhi Capitals,6.763269,"[Kuldeep Yadav, Mohit Sharma, Ashutosh Sharma,..."
7,Lucknow Super Giants,5.910746,"[Mayank Yadav, Ravi Bishnoi, Himmat Singh, Sha..."
8,Sunrisers Hyderabad,5.880838,"[Pat Cummins, Abhishek Sharma, Rahul Chahar, M..."
9,Punjab Kings,5.185222,"[Glenn Maxwell, Marcus Stoinis, Yuzvendra Chah..."
